# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("../output_data/output_data_file")
del city_df["Unnamed: 0"]
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nyurba,63.2842,118.3319,50.32,64,60,6.73,RU,1620191762
1,Lompoc,34.6391,-120.4579,53.01,87,2,4.56,US,1620191763
2,Rikitea,-23.1203,-134.9692,77.32,82,100,5.70,PF,1620191763
3,Ylöjärvi,61.5563,23.5961,39.99,56,0,11.50,FI,1620191764
4,Qaanaaq,77.4840,-69.3632,23.99,68,100,6.38,GL,1620191764
...,...,...,...,...,...,...,...,...,...
553,Gua Musang,4.8823,101.9644,83.89,78,99,2.44,MY,1620192017
554,Phetchabun,16.2500,101.0833,95.00,52,20,2.30,TH,1620192018
555,Sibu,2.3000,111.8167,86.00,70,40,4.61,MY,1620192018
556,Jacareacanga,-6.2222,-57.7528,71.73,98,44,2.35,BR,1620192019


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# define humidity
humidity = city_df["Humidity"].astype(float)

# define locations for Lat and Lng
lat_lng = city_df[["Lat", "Lng"]]

In [5]:
# create a gmap figure
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_lng, weights=humidity, dissipating=False, point_radius=5, max_intensity=100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_df = city_df.loc[(city_df["Max Temp"] > 65)
                         & (city_df["Max Temp"] < 85)
                         & (city_df["Wind Speed"] < 10)
                         & (city_df["Cloudiness"] < 10) 
                         & (city_df["Humidity"] < 50), :]
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,Safaga,26.7292,33.9365,84.20,12,0,6.91,EG,1620191769
85,Dārāb,28.7519,54.5444,74.68,19,1,2.75,IR,1620191806
97,Buckeye,33.3703,-112.5838,78.80,13,3,6.91,US,1620191700
112,Ceres,37.5949,-120.9577,75.20,31,1,8.05,US,1620191816
194,Saint George,37.1041,-113.5841,72.00,15,1,3.44,US,1620191834
244,Baijiantan,45.6333,85.1833,76.44,9,0,7.49,CN,1620191872
252,Korla,41.7597,86.1469,74.28,16,0,2.39,CN,1620191877
299,Sakakah,29.9697,40.2064,82.40,14,0,4.61,SA,1620191899
324,Zaysan,47.4667,84.8667,68.20,20,0,6.76,KZ,1620191913
325,Rūdbār,30.1500,62.6000,82.06,30,2,4.25,AF,1620191914


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create new df to store into variable named hotel_df
hotel_df = vacation_df

In [8]:
# loop through df and use google API
for index, row in hotel_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        # make parameter dictionary
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        
        latitude = row["Lat"]
        longitude = row["Lng"]

        params["location"] = f"{latitude}, {longitude}"
        
        # request from google API
        hotel_data = requests.get(url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    
    # get rid of any NaN data before showing new dataframe   
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

/Users/cheyennemartin/opt/anaconda3/envs/EnvMod/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/cheyennemartin/opt/anaconda3/envs/EnvMod/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,Safaga,26.7292,33.9365,84.20,12,0,6.91,EG,1620191769,Amarina Hotel e resort
85,Dārāb,28.7519,54.5444,74.68,19,1,2.75,IR,1620191806,Naghshe shapoor Hotel
97,Buckeye,33.3703,-112.5838,78.80,13,3,6.91,US,1620191700,Buckeye Motor Hotel
112,Ceres,37.5949,-120.9577,75.20,31,1,8.05,US,1620191816,DoubleTree by Hilton Hotel Modesto
194,Saint George,37.1041,-113.5841,72.00,15,1,3.44,US,1620191834,Comfort Inn Saint George North
244,Baijiantan,45.6333,85.1833,76.44,9,0,7.49,CN,1620191872,NaN
252,Korla,41.7597,86.1469,74.28,16,0,2.39,CN,1620191877,NaN
299,Sakakah,29.9697,40.2064,82.40,14,0,4.61,SA,1620191899,ZAYAFAH AL RIMAL hotel
324,Zaysan,47.4667,84.8667,68.20,20,0,6.76,KZ,1620191913,Gostinitsa Aray
325,Rūdbār,30.1500,62.6000,82.06,30,2,4.25,AF,1620191914,NaN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))